In [30]:

'''import all the required modules'''

import os
import pandas as pd
import regex as re
from tqdm.notebook import tqdm
import PIL
import PIL.Image
import matplotlib.pyplot as plt
import math
import numpy as np
from scipy.ndimage.filters import gaussian_filter
from tqdm.notebook import tqdm

session_number = "001"
participant_name = "trial8"

'''resolution of the screen , device where eye tracking test takes place'''

screen_width = 1920
screen_height = 1080

''' df_eyetracking - - - Contains participants's eye tracking data.
    l_valid        - - - left_gaze_point_validity.
    r_valid        - - - right_gaze_point_validity.
    l_display      - - - left_gaze_point_on_display_area. 
    r_display      - - - right_gaze_point_on_display_area.
    time           - - - system_time_stamp.
    l_diameter     - - - left_pupil_diameter.
    r_diameter     - - - right_pupil_diameter.
    
    df_answers     - - - Contains answers given by participant.
    df_cutting     - - - Contains time spent on each images. '''

df_eyetracking = pd.read_csv("./data/eye_tracking_data_" + session_number + " _ " + participant_name + ".csv", sep=";")
df_eyetracking.columns = ["l_valid", "r_valid", "l_display", "r_display", "time", "l_diameter", "r_diameter"]
df_answers = pd.read_csv("./data/study_answers_" + session_number + " _ " + participant_name + ".csv")
df_answers = df_answers.iloc[:, 1:]
df_cutting = pd.read_csv("./data/study_cutting_data_" + session_number + " _ " + participant_name + ".csv")
df_cutting = df_cutting.iloc[:, 1:]


df_eyetracking["l_display_x"] = df_eyetracking["l_display"].apply(lambda row: int(float(re.findall("\d+\.\d+", row)[0])*screen_width) if len(re.findall("\d+\.\d+", row)) == 2 else 0.0)
df_eyetracking["l_display_y"] = df_eyetracking["l_display"].apply(lambda row: int(float(re.findall("\d+\.\d+", row)[1])*screen_height) if len(re.findall("\d+\.\d+", row)) == 2 else 0.0)
df_eyetracking["r_display_x"] = df_eyetracking["r_display"].apply(lambda row: int(float(re.findall("\d+\.\d+", row)[0])*screen_width) if len(re.findall("\d+\.\d+", row)) == 2 else 0.0)
df_eyetracking["r_display_y"] = df_eyetracking["r_display"].apply(lambda row: int(float(re.findall("\d+\.\d+", row)[1])*screen_height) if len(re.findall("\d+\.\d+", row)) == 2 else 0.0)

df_eyetracking = df_eyetracking.drop("l_display", axis=1)
df_eyetracking = df_eyetracking.drop("r_display", axis=1)





In [31]:
#time conversion
eye_start = df_eyetracking.loc[0, "time"]
df_eyetracking["time"] = df_eyetracking["time"] - eye_start
df_eyetracking["time"] = df_eyetracking["time"] / 1000000

In [ ]:

#heatmap Generation

def gen_heatmap(click_data, width, height, radius):
    heatmap = np.zeros((height, width), dtype=np.int32)
    mask = np.ones((height, width), dtype=np.int32)
    
    for x, y in tqdm(click_data):
        Y, X = np.ogrid[:height, :width]
        dist_from_center = np.sqrt((X - x)**2 + (Y-y)**2)
        current_mask = dist_from_center < radius
        heatmap[current_mask] = heatmap[current_mask] + mask[current_mask]
        
    return heatmap

dimension = (1080,1920)

number_of_clicks = 1000

time_filter=df_eyetracking["time"][0]

#iterating over images and thier start and end time

for idx, row in tqdm(df_cutting.iterrows(), total=len(df_cutting)):
    image = row["ImagePath"]
    start_time = row["StartTime"]
    end_time = row["EndTime"]
    
    df_tmp = df_eyetracking[(df_eyetracking["time"] >= start_time) & (df_eyetracking["time"] <= end_time)]
    
    #accepting only valid values form eye tracking data
    
    x_values_l = df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)]["l_display_x"].values
    y_values_l = df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)] ["l_display_y"].values

    x_values_r = df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)]["r_display_x"].values
    y_values_r = df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)]["r_display_y"].values

    #taking average of x_value_l,x_value_r
    
    x_values = [ int((x_l+x_r)/2.0) for (x_l, x_r) in zip(x_values_l, x_values_r)]
    y_values = [ int((y_l+y_r)/2.0) for (y_l, y_r) in zip(y_values_l, y_values_r)] 
    #click data is average of both x and y coordinate for left eye and right eye

    click_data = list(zip([int(x) for x in x_values], [int(y) for y in y_values]))
    
    #image = image.replace("./snippets/Final","./snippets")
    imageName = image
    image = PIL.Image.open(imageName)
    w, h = image.size
    heatmap= gen_heatmap(click_data, w, h, 50)
    
    
    #perform gaussian blur
    cmap = plt.cm.get_cmap("jet") 
    heatmap = gaussian_filter(heatmap, sigma=5)
    heatmap = cmap(heatmap)
    plt.imshow(heatmap, cmap='jet', interpolation='nearest')
    image.resize((h, w))
    image = np.array(image)
    heatmap=heatmap*255
    heatmap=heatmap.astype(int)
    heatmap_image =0.7*image + 0.3*heatmap
    heatmap_image = heatmap_image
    algorithm_name = imageName.split("/")[-1]
    image_name = f"./{participant_name}/heatmap_{algorithm_name}"
    im = PIL.Image.fromarray(np.uint8(heatmap_image))
    im.save(image_name)
    plt.close('all')

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/1698 [00:00<?, ?it/s]

  0%|          | 0/2065 [00:00<?, ?it/s]

  0%|          | 0/2423 [00:00<?, ?it/s]

  0%|          | 0/10956 [00:00<?, ?it/s]

In [6]:
df_cutting

,ImagePath,StartTime,EndTime,ImageTime
0,./snippets/Final/ArraySum(CNL).png,2.209393,21.450616,19.241223
1,./snippets/Final/ArraySumresp(CNL).png,21.455601,47.884162,26.428561
2,./snippets/Final/BubbleSort(WIC).png,49.671287,78.149696,28.478409
3,./snippets/Final/BubbleSortresp(WIC).png,78.154684,207.010952,128.856268
4,./snippets/Final/NumberCheck(WOC).png,209.430845,223.678388,14.247543
5,./snippets/Final/NumberCheckresp(WOC).png,223.682376,262.228537,38.546161
6,./snippets/Final/ContainsSubstring(CNL).png,263.213113,291.690168,28.477055
7,./snippets/Final/ContainsSubstringresp(CNL).png,291.694190,306.693061,14.998871
8,./snippets/Final/Power(WIC).png,308.178412,320.608164,12.429752
9,./snippets/Final/Powerresp(WIC).png,320.612151,345.090010,24.477859
